# Day 02.12.2021 Simulation

In [1]:
import pandas as pd
import numpy as np
from data_tools.simulation_data import SimulationDataCreator
from data_tools.simulation_run import RunSimulation
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.precision", 5)
# sha1:a5b97801e15f:63b102aeba2a3f7a1b44c8fbd715ecd6e17abde3

In [2]:
# upload, clean and prepare the data
simulation_data_creator = SimulationDataCreator("/Users/tom/dev/bbp-research/configuration_files/simulation_data_creator.json")
# upload the data
data_dict = simulation_data_creator.upload_data(["market_data", "market_data_1", "view_market_data"])
# convert invoker column
data_dict = simulation_data_creator.convert_invokers(data_dict, ["market_data", "market_data_1", "view_market_data"])
# merge the data
merged_data = simulation_data_creator.merge_markets(data_dict["market_data"]["data"], data_dict["market_data_1"]["data"], "timestamp", True)
merged_data = simulation_data_creator.merge_markets(merged_data, data_dict["view_market_data"]["data"], "timestamp", True)
# # add time
merged_data = simulation_data_creator.add_time(merged_data, "timestamp")
# data_dict = simulation_data_creator.add_times(data_dict, ["market_data", "market_data_1", "view_market_data"])
# cut hours in between 10:00 to 19:00
merged_data = simulation_data_creator.keep_data_between_hours(merged_data, 9, 19, "hour")

the value counts of column invoker is:

FCI_offer_0    720757
FCI_bid_0      683806
IBI_bid_0      294108
IBI_offer_0    294108
TA1_offer_0    183453
TA1_bid_0      183452
EBS_offer_0    170324
EBS_bid_0      170324
IDB_offer_0    154616
IDB_bid_0      154616
dtype: int64
1705224, 1705224
the value counts of column invoker is:

CITIQ_BARAK_offer_0       44581
CITIQ_BARAK_bid_0         44534
CREDITQ_BARAK_offer_0     34838
CREDITQ_BARAK_bid_0       34831
JPQ_BARAK_bid_0           32918
JPQ_BARAK_offer_0         30401
UBSQ_BARAK_offer_0        23504
UBSQ_BARAK_bid_0          23481
GOLDMANQ_BARAK_bid_0      21705
GOLDMANQ_BARAK_offer_0    21661
dtype: int64
285184, 285184
the value counts of column invoker is:

TRADAIR_LP12_offer_0    218767
TRADAIR_LP12_bid_0      218767
TRADAIR_LP1_offer_0     154098
TRADAIR_LP1_bid_0       154098
TRADAIR_LP15_bid_0      133454
TRADAIR_LP15_offer_0    133454
TRADAIR_LP3_bid_0       118637
TRADAIR_LP3_offer_0     118637
TRADAIR_LP6_bid_0       116233
TRA

## arena 1 simulation 1
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005
- hit_brokers = ['FCI', 'IBI', 'TA1', 'IDB']
- view_brokers = none
- positon_fee = 0.00001
- max time in position = false

In [3]:
arena_1_sim_1 = RunSimulation(simulation_name = "arena 1 simulation 1", 
                              hit_brokers = ['FCI', 'IBI', 'TA1', 'IDB'],
                              view_brokers = [],
                              enter_conditions = "spread_tag_inc(TA1_offer, IBI_bid) < 0",
                              exit_conditions = "current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005",
                              positon_fee = 0.00001, 
                              max_time_in_position = False)
# initialize summary data frame                   
arena_1_sim_1.initialize_summary_df("increase")
# prepaer data for simulation
arena_1_sim_1_df = merged_data.copy()
sim_df_col_list, sim_df_np, sim_number_of_rows = arena_1_sim_1.prepaer_data_for_simulation(arena_1_sim_1_df)

In [4]:
arena_1_sim_1.initialize_summary_df("increase")
for i in range(sim_number_of_rows):
    # current market row
    current_market_data = sim_df_np[i]
    # check if entering condition holds
    if current_market_data[sim_df_col_list.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_1_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            upper_limit_value = best_offer_value + 0.0001
            lower_limit_value = best_bid_value - 0.00005
            while temp_index < sim_number_of_rows and sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] < upper_limit_value and sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] > lower_limit_value:
                temp_index += 1
            if temp_index >= sim_number_of_rows:
                break
            else:
                arena_1_sim_1.documnet_position(current_market_data, sim_df_np[temp_index], spread_tag_inc)

In [5]:
df_to_present = arena_1_sim_1.create_simulation_summary()
df_to_present
print(f"")
print(df_to_present.shape[0])
print((df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]).describe())
print(df_to_present["position_profit"].sum())

print((df_to_present['position_profit'] > 0).sum()/ df_to_present.shape[0])


2949
count      2949.00000
mean      14571.82943
std       21776.49608
min           1.00000
25%         253.00000
50%        5530.00000
75%       19967.00000
max      250446.00000
dtype: float64
0.1562
0.6785350966429298


In [13]:
arena_1_sim_1_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2068387 entries, 289573 to 2877652
Data columns (total 263 columns):
 #    Column                                     Dtype         
---   ------                                     -----         
 0    timestamp                                  int64         
 1    TA1_offer_0_timestamp                      float64       
 2    TA1_offer_0_rate                           float64       
 3    TA1_offer_0_bbp_timestamp                  float64       
 4    TA1_offer_0_size                           float64       
 5    TA1_offer_0_original_timestamp             float64       
 6    TA1_bid_0_timestamp                        float64       
 7    TA1_bid_0_rate                             float64       
 8    TA1_bid_0_bbp_timestamp                    float64       
 9    TA1_bid_0_size                             float64       
 10   TA1_bid_0_original_timestamp               float64       
 11   FCI_offer_0_timestamp                      

In [ ]:
lps_name_list = ["TRADAIR_LP1","TRADAIR_LP2","TRADAIR_LP3", "TRADAIR_LP4","TRADAIR_LP5","TRADAIR_LP6","TRADAIR_LP7","TRADAIR_LP12","TRADAIR_LP13", "TRADAIR_LP14", "TRADAIR_LP15"]
offer_lps_name_list = []
bid_lps_name_list = []
for lp_name in lps_name_list:
    lp_offer_name_to_add = f"{lp_name}_offer_0"
    offer_lps_name_list.append(lp_offer_name_to_add)
    lp_bid_name_to_add = f"{lp_name}_bid_0"
    bid_lps_name_list.append(lp_bid_name_to_add)
print(offer_lps_name_list)
print(bid_lps_name_list)

horses_dict = {}
for offer_lp in offer_lps_name_list:
    for bid_lp in bid_lps_name_list:
        hores_entery = f"{offer_lp}_s_{bid_lp}"
        horses_dict[hores_entery] = {"value": np.NaN, "valid": False}
        # horses_dict[hores_entery] = np.NaN
print(horses_dict)

In [67]:

def create_feat_dict_and_policy(relevent_brokers):
    # create feat policy
    feat_policy = {}
    for broker_bane in relevent_brokers:
        invoker_bid_col_name = f"{broker_bane}_bid_0"
        feat_policy[invoker_bid_col_name] = []
        invoker_offer_col_name = f"{broker_bane}_offer_0"
        feat_policy[invoker_offer_col_name] = []
        
    # create feat dict
    feat_dict = {}

    # create spread tag TA1 offer spread tag IBI bid
    feat_dict["TA1_offer_0_st_IBI_bid_0"] = {"value": np.NaN, "last_change": "", "args": ["TA1_offer_0_rate", "IBI_bid_0_rate"]}
    # add to policy
    feat_policy["TA1_offer_0"].append("TA1_offer_0_st_IBI_bid_0")
    feat_policy["IBI_bid_0"].append("TA1_offer_0_st_IBI_bid_0")


    # create spread tag LP's
    lps_name_list = ["TRADAIR_LP1","TRADAIR_LP2","TRADAIR_LP3", "TRADAIR_LP4","TRADAIR_LP5","TRADAIR_LP6","TRADAIR_LP7","TRADAIR_LP12","TRADAIR_LP13", "TRADAIR_LP14", "TRADAIR_LP15"]
    LPs_st_list = []
    for lp_name in lps_name_list:
        temp_lp_name_list = lps_name_list.copy()
        temp_lp_name_list.remove(lp_name)
        lp_offer_name_to_add = f"{lp_name}_offer_0"
        for other_lp_name in temp_lp_name_list:
            lp_bid_name_to_add = f"{other_lp_name}_bid_0"
            entery_to_create = f"{lp_offer_name_to_add}_st_{lp_bid_name_to_add}"
            lp_offer_name_arg = f"{lp_offer_name_to_add}_rate"
            lp_bid_name_arg = f"{lp_bid_name_to_add}_rate"
            feat_dict[entery_to_create] = {"value": np.NaN, "last_change": "", "args": [lp_offer_name_arg, lp_bid_name_arg]}
            feat_policy[lp_offer_name_to_add].append(entery_to_create)
            feat_policy[lp_bid_name_to_add].append(entery_to_create)
            LPs_st_list.append(entery_to_create)
    
    return feat_dict, feat_policy, LPs_st_list

def craeate_relevent_invokers_columns(relevent_brokers):
    bid_invoker_columns = []
    offer_invoker_columns = []
    for broker_bane in relevent_brokers:
        invoker_bid_col_name = f"{broker_bane}_bid_0"
        bid_invoker_columns.append(invoker_bid_col_name)
        invoker_offer_col_name = f"{broker_bane}_offer_0"
        offer_invoker_columns.append(invoker_offer_col_name)
    return bid_invoker_columns, offer_invoker_columns


In [68]:
# create relevent ionvokers for sim 1
hit_brokers = ['FCI', 'IBI', 'TA1', 'IDB']
lps_name_list = ["TRADAIR_LP1","TRADAIR_LP2","TRADAIR_LP3", "TRADAIR_LP4","TRADAIR_LP5","TRADAIR_LP6","TRADAIR_LP7","TRADAIR_LP12","TRADAIR_LP13", "TRADAIR_LP14", "TRADAIR_LP15"]
sim_1_relevent_brokers = list(set().union(hit_brokers, lps_name_list))
sim_1_bid_relevent_brokers, sim_1_offer_relevent_brokers = craeate_relevent_invokers_columns(sim_1_relevent_brokers)
feat_dict, feat_policy, horses_feat_list = create_feat_dict_and_policy(sim_1_relevent_brokers)


In [69]:
def cal_feat(feat_dict, feat_policy, current_market_data, sim_col_name_list, bid_relevent_brokers_to_update_on, offer_relevent_brokers_to_update_on):
    # itterate over True bid invokers
    for bid_relevent_broker in bid_relevent_brokers_to_update_on:
        # check if they invoke
        if current_market_data[sim_col_name_list.index(bid_relevent_broker)]:
            # update the feat in which the invoker toke place
            for feat_to_update in feat_policy[bid_relevent_broker]:
                offer_arg = feat_dict[feat_to_update]["args"][0]
                bid_arg = feat_dict[feat_to_update]["args"][1]
                feat_dict[feat_to_update]["value"] = np.round(current_market_data[sim_col_name_list.index(offer_arg)] - current_market_data[sim_col_name_list.index(bid_arg)], 6)
                feat_dict[feat_to_update]["last_change"] = "bid"
    # itterate over True offer invokers
    for offer_relevent_broker in offer_relevent_brokers_to_update_on:
        # check if they invoke
        if current_market_data[sim_col_name_list.index(offer_relevent_broker)]:
            # update the feat in which the invoker toke place
            for feat_to_update in feat_policy[offer_relevent_broker]:
                offer_arg = feat_dict[feat_to_update]["args"][0]
                bid_arg = feat_dict[feat_to_update]["args"][1]
                feat_dict[feat_to_update]["value"] = np.round(current_market_data[sim_col_name_list.index(offer_arg)] - current_market_data[sim_col_name_list.index(bid_arg)], 6)
                feat_dict[feat_to_update]["last_change"] = "offer"
    return feat_dict

In [ ]:
feat_dict = cal_feat(feat_dict, feat_policy, sim_df_np[341], sim_df_col_list, sim_1_bid_relevent_brokers, sim_1_offer_relevent_brokers)
print(feat_dict)

In [92]:
arena_1_sim_1.initialize_summary_df("increase")
# itterate on every row
# for i in range(sim_number_of_rows):
# for i in range(1034193, 1044193):
for i in range(1004193, 1014193):
    # cut current market data
    current_market_data = sim_df_np[i]
    # update feats
    feat_dict = cal_feat(feat_dict, feat_policy, current_market_data, sim_df_col_list, sim_1_bid_relevent_brokers, sim_1_offer_relevent_brokers)
    # check if condition holds
    if feat_dict["TA1_offer_0_st_IBI_bid_0"]["last_change"] == "bid" and feat_dict["TA1_offer_0_st_IBI_bid_0"]["value"] < 0.0:
        temp_v = feat_dict["TA1_offer_0_st_IBI_bid_0"]["value"]
        print(f"{temp_v} has horeses:")
        number_of_horeses = 0
        for lp_st in horses_feat_list:
            # if feat_dict[lp_st]["value"] < 0.0:
            if feat_dict[lp_st]["last_change"] == "bid" and feat_dict[lp_st]["value"] <= -0.00001:
                number_of_horeses += 1
                # print(feat_dict[lp_st]["value"])
                # print(current_market_data)
        print(number_of_horeses)
        print(" ")
        
    
    


-0.00017 has horeses:
21
 


In [89]:
sim_number_of_rows /2

1034193.0

In [25]:
# with horese
arena_1_sim_1.initialize_summary_df("increase")
for i in range(sim_number_of_rows):
    # current market row
    current_market_data = sim_df_np[i]
    # calculate number of active horese in offer direction
    for lp_name in lps_name_list:
        # offer side
        lp_offer = f"{lp_name}_offer_0"
        if current_market_data[sim_df_col_list.index(lp_offer)]:
            lp_bid = f"{lp_name}_bid_0"
            temp_brokers = bid_lps_name_list.copy()
            temp_brokers.remove(lp_bid)
            for temp_lp_bid in temp_brokers:
                entery_to_update = f"{lp_offer}_s_{temp_lp_bid}"
                offer_rate = f"{lp_offer}_rate"
                bid_rate = f"{temp_lp_bid}_rate"
                horses_dict[entery_to_update]["value"] = np.round(current_market_data[sim_df_col_list.index(offer_rate)] - current_market_data[sim_df_col_list.index(bid_rate)], 6)
                horses_dict[entery_to_update]["valid"] = False
        # bid side
        lp_bid = f"{lp_name}_bid_0"
        if current_market_data[sim_df_col_list.index(lp_bid)]:
            lp_offer = f"{lp_name}_offer_0"
            temp_brokers = offer_lps_name_list.copy()
            temp_brokers.remove(lp_offer)
            for temp_lp_offer in temp_brokers:
                entery_to_update = f"{temp_lp_offer}_s_{lp_bid}"
                offer_rate = f"{temp_lp_offer}_rate"
                bid_rate = f"{lp_bid}_rate"
                horses_dict[entery_to_update]["value"] = np.round(current_market_data[sim_df_col_list.index(offer_rate)] - current_market_data[sim_df_col_list.index(bid_rate)], 6)
                horses_dict[entery_to_update]["valid"] = True

    # check if entering condition holds
    if current_market_data[sim_df_col_list.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            # print ture horese with value less or equal to -0.0001
            horese_list = []
            for key, value in horses_dict.items():
                if value["valid"] and value["value"] <= -0.00001:
                    hores_value = value["value"]
                    horese_list.append(f"{key}_{hores_value}")
            print(horese_list)
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_1_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            upper_limit_value = best_offer_value + 0.0001
            lower_limit_value = best_bid_value - 0.00005
            while temp_index < sim_number_of_rows and sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] < upper_limit_value and sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] > lower_limit_value:
                temp_index += 1
            if temp_index >= sim_number_of_rows:
                break
            else:
                arena_1_sim_1.documnet_position(current_market_data, sim_df_np[temp_index], spread_tag_inc)

TRADAIR_LP13_offer_0_s_TRADAIR_LP7_bid_0
TRADAIR_LP13_offer_0_s_TRADAIR_LP7_bid_0
TRADAIR_LP2_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP3_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP4_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP7_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP14_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP2_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP3_offer_0_s_TRADAIR_LP14_bid_0
TRADAIR_LP7_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP12_offer_0_s_TRADAIR_LP14_bid_0
TRADAIR_LP13_offer_0_s_TRADAIR_LP14_bid_0
TRADAIR_LP2_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP7_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP12_offer_0_s_TRADAIR_LP14_bid_0
TRADAIR_LP13_offer_0_s_TRADAIR_LP14_bid_0
TRADAIR_LP2_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP7_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP12_offer_0_s_TRADAIR_LP14_bid_0
TRADAIR_LP13_offer_0_s_TRADAIR_LP14_bid_0
TRADAIR_LP2_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP7_offer_0_s_TRADAIR_LP5_bid_0
TRADAIR_LP12_offer_0_s_TRADAIR_LP14_bid_0
TRADAIR_LP13_offer_0_s_TRADAIR_LP14_bid_0
TRADAIR_LP7_offer_0_

In [20]:
list1 = ['a', 'b', 'c']
list2 = list1.copy()
list2.remove('a')
print(list2)

['b', 'c']


In [ ]:
df_to_present.head(10)

In [ ]:
# check time differnce between signal and best change
# arena_1_sim_1.initialize_summary_df("increase")
signal_entering_time = []
best_change_time = []
best_offer_at_signal = []
best_offer_after_first_change = []
for i in range(sim_number_of_rows):
    # current market row
    current_market_data = sim_df_np[i]
    # check if entering condition holds
    if current_market_data[sim_df_col_list.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_1_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            while temp_index < sim_number_of_rows and sim_df_np[temp_index][sim_df_col_list.index("best_offer_hit_rate")] == best_offer_value:
                temp_index += 1
            if temp_index >= sim_number_of_rows:
                break
            else:
                signal_entering_time.append(current_market_data[sim_df_col_list.index("timestamp")])
                best_change_time.append(sim_df_np[temp_index][sim_df_col_list.index("timestamp")])
                best_offer_at_signal.append(current_market_data[sim_df_col_list.index("best_offer_hit_rate")])
                best_offer_after_first_change.append(sim_df_np[temp_index][sim_df_col_list.index("best_offer_hit_rate")])

signal_to_best_change_in_offer_summary = {
    "signal_entering_time": signal_entering_time,
    "best_change_time": best_change_time,
    "best_offer_at_signal": best_offer_at_signal,
    "best_offer_after_first_change": best_offer_after_first_change
}
signal_to_best_change_in_offer_df = pd.DataFrame.from_dict(signal_to_best_change_in_offer_summary)
signal_to_best_change_in_offer_df["diff"] = signal_to_best_change_in_offer_df["best_change_time"] - signal_to_best_change_in_offer_df["signal_entering_time"]

In [ ]:
signal_to_best_change_in_offer_df["best_change_time"].value_counts()

In [ ]:
signal_to_best_change_in_offer_df.loc[2750:2800]

In [ ]:
signal_to_best_change_in_offer_df["diff"].describe()

In [ ]:
path = "/Users/tom/dev/data/bpp_servers/server7/06-12-2021/eurusd.txt"
temp_raw_data = pd.read_csv(path, names=["timestamp", "bank_name", "quote_type", "quote_value", "quote_size", "bbp_timestamp", "level", "ebs_timestamp"])
temp_raw_data.head(10)

In [ ]:
temp_raw_data.loc[1083954:1083984]


In [ ]:
temp_raw_data[temp_raw_data["ebs_timestamp"] == 1638802800123]

In [ ]:
temp_raw_data[temp_raw_data["ebs_timestamp"] == 1638802800225]


In [ ]:
temp_raw_data[temp_raw_data["ebs_timestamp"] == 1638802800425]

In [ ]:
temp_raw_data.loc[1594727: 1594765]

In [ ]:
(temp_raw_data["ebs_timestamp"]%100).value_counts()

In [ ]:
temp_raw_data[temp_raw_data["ebs_timestamp"] == 11638802800625]

In [ ]:
1638802800425 - 1638802800123

## arena 2 simulation 1
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005
- hit_brokers = ["CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"]
- view_brokers = [TA1, IBI]
- positon_fee = 0.00001
- max time in position = false

In [ ]:
arena_2_sim_1 = RunSimulation(simulation_name = "arena 2 simulation 1", 
                              hit_brokers = ["CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"],
                              view_brokers = [],
                              enter_conditions = "spread_tag_inc(TA1_offer, IBI_bid) < 0",
                              exit_conditions = "current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005",
                              positon_fee = 0.00001, 
                              max_time_in_position = False)
# initialize summary data frame                   
arena_2_sim_1.initialize_summary_df("increase")
# prepaer data for simulation
arena_2_sim_1_df = merged_data.copy()
sim_df_col_list2, sim_df_np2, sim_number_of_rows2 = arena_2_sim_1.prepaer_data_for_simulation(arena_2_sim_1_df)

In [ ]:
arena_2_sim_1.initialize_summary_df("increase")
for i in range(sim_number_of_rows2):
    # current market row
    current_market_data = sim_df_np2[i]
    # check if entering condition holds
    if current_market_data[sim_df_col_list2.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list2.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list2.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_2_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            upper_limit_value = best_offer_value + 0.0003
            lower_limit_value = best_bid_value - 0.0001
            # while temp_index < sim_number_of_rows and (sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] >= upper_limit_value or sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] < lower_limit_value):
            while temp_index < sim_number_of_rows2 and sim_df_np2[temp_index][sim_df_col_list2.index("best_bid_hit_rate")] < upper_limit_value and sim_df_np2[temp_index][sim_df_col_list2.index("best_bid_hit_rate")] > lower_limit_value:
                temp_index += 1
            if temp_index >= sim_number_of_rows2:
                break
            else:
                arena_2_sim_1.documnet_position(current_market_data, sim_df_np2[temp_index], spread_tag_inc)

In [ ]:
# limits + 0.0003 -0.0001
df_to_present = arena_2_sim_1.create_simulation_summary()
# df_to_presen

print(df_to_present.shape[0])
print((df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]).describe())
print(df_to_present.shape[0])
print(df_to_present["position_profit"].sum())

In [ ]:
# limits + 0.0002 -0.00005
df_to_present = arena_2_sim_1.create_simulation_summary()
# df_to_presen

print(df_to_present.shape[0])
print((df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]).describe())
print(df_to_present.shape[0])
print(df_to_present["position_profit"].sum())

In [ ]:
# limits + 0.0001 -0.00005
df_to_present = arena_2_sim_1.create_simulation_summary()
# df_to_presen

print(df_to_present.shape[0])
print((df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]).describe())
print(df_to_present.shape[0])
print(df_to_present["position_profit"].sum())

In [ ]:
df_to_present["diff"] = df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]
df_to_present["diff"].describe()

In [ ]:
df_to_present["position_profit"].sum()

## arena 3 simulation 1
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005
- hit_brokers = ["FCI", "IBI", "TA1", "IDB", "CITIQ_BARAK", "CREDITQ_BARAK","UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"]
- view_brokers = None
- positon_fee = 0.00001
- max time in position = false

In [ ]:
arena_3_sim_1 = RunSimulation(simulation_name = "arena 3 simulation 1", 
                              hit_brokers = ["FCI", "IBI", "TA1", "IDB", "CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"],
                              view_brokers = [],
                              enter_conditions = "spread_tag_inc(TA1_offer, IBI_bid) < 0",
                              exit_conditions = "current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005",
                              positon_fee = 0.00001, 
                              max_time_in_position = False)
# initialize summary data frame                   
arena_3_sim_1.initialize_summary_df("increase")
# prepaer data for simulation
arena_3_sim_1_df = merged_data.copy()
sim_df_col_list3, sim_df_np3, sim_number_of_rows3 = arena_3_sim_1.prepaer_data_for_simulation(arena_3_sim_1_df)

In [ ]:
for i in range(sim_number_of_rows3):
    # current market row
    current_market_data = sim_df_np3[i]
    # check if entering condition holds
    if current_market_data[sim_df_col_list3.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list3.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list3.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_3_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            upper_limit_value = best_offer_value + 0.0001
            lower_limit_value = best_bid_value - 0.0001
            # while temp_index < sim_number_of_rows and (sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] >= upper_limit_value or sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] < lower_limit_value):
            while temp_index < sim_number_of_rows3 and sim_df_np3[temp_index][sim_df_col_list3.index("best_bid_hit_rate")] < upper_limit_value and sim_df_np3[temp_index][sim_df_col_list3.index("best_bid_hit_rate")] > lower_limit_value:
                temp_index += 1
            if temp_index >= sim_number_of_rows3:
                break
            else:
                arena_3_sim_1.documnet_position(current_market_data, sim_df_np3[temp_index], spread_tag_inc)

In [ ]:
df_to_present = arena_3_sim_1.create_simulation_summary()
df_to_present

In [ ]:
df_to_present["diff"] = df_to_present["timestamp_at_exiting"] - df_to_present["timestamp_at_entering"]
df_to_present["diff"].describe()

In [ ]:
df_to_present["position_profit"].sum()

In [ ]:
df_to_present.shape[0]

## arena 1 simulation 2
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005 AND 3 horese in the offer direction
- hit_brokers = ['FCI', 'IBI', 'TA1', 'IDB']
- view_brokers = LP's
- positon_fee = 0.00001
- max time in position = false

## arena 2 simulation 2
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005 AND 3 horese in the offer direction
- hit_brokers = ["CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"]
- view_brokers = [TA1, IBI] + LP's
- positon_fee = 0.00001
- max time in position = false